In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import glob
tf.logging.set_verbosity(tf.logging.ERROR)
import numpy as np
from tensorboardX import SummaryWriter

In [2]:
# from youtube-8m utils.py
def Dequantize(feat_vector, max_quantized_value=2, min_quantized_value=-2):
  """Dequantize the feature from the byte format to the float format.

  Args:
    feat_vector: the input 1-d vector.
    max_quantized_value: the maximum of the quantized value.
    min_quantized_value: the minimum of the quantized value.

  Returns:
    A float vector which has the same shape as feat_vector.
  """
  assert max_quantized_value > min_quantized_value
  quantized_range = max_quantized_value - min_quantized_value
  scalar = quantized_range / 255.0
  bias = (quantized_range / 512.0) + min_quantized_value
  return feat_vector * scalar + bias

def decode(feat_vector, feature_size):
    return tf.reshape(tf.cast(tf.decode_raw(feat_vector, 
                                            tf.uint8), 
                              tf.float32),
                      [-1, feature_size])

In [240]:
# filepath is path to tfrecord
# datatype is audio or video
# output_features and output_labels are empty lists or existing lists
def load_data(filepath, data_type, output_labels, output_features):
    if data_type == 'audio':
        context = {
            'labels': tf.VarLenFeature(dtype=tf.int64)
        }

        feature_list = {
            'audio_embedding': tf.FixedLenSequenceFeature([], dtype=tf.string)
        }
        feature_name = 'audio_embedding'
        feature_len = 128

    elif data_type == 'video':
        context = {
            'id': tf.FixedLenFeature([], dtype=tf.string),
            'labels': tf.VarLenFeature(dtype=tf.int64)
        }

        feature_list = {
            'rgb': tf.FixedLenSequenceFeature([], dtype=tf.string),
        }
        feature_name = 'rgb'
        feature_len = 128
        
        
    tf.reset_default_graph()    


    # Read TFRecord file
    reader = tf.TFRecordReader()
    filename_queue = tf.train.string_input_producer([filepath])


    # Extract features from serialized data

    _, serialized_example = reader.read(filename_queue)
    context, features = tf.io.parse_single_sequence_example(serialized_example,
                                                    context_features=context,
                                                    sequence_features=feature_list,
                                                    example_name=None,
                                                    name=None
    )
    labels = context['labels']
    label = labels.values[0]
    data = Dequantize(decode(features[feature_name], feature_len))

    # Many tf.train functions use tf.train.QueueRunner,
    # so we need to start it before we read
    
    
    with tf.Session() as sess:
        sess.run(tf.local_variables_initializer())
        sess.run(tf.global_variables_initializer())
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        # f = codecs.open(outpath, "w", encoding='utf-8')
        try:
            counter = 0
            recordlist = []

            num_in_file = sum(1 for _ in tf.python_io.tf_record_iterator(filepath))

            for i in range(num_in_file):
                d, l = sess.run([data, label])
                output_labels.append(l)
                output_features.append(d)
        except tf.errors.OutOfRangeError:
            print('Finished extracting from tfrecord data.')
        finally:
            coord.request_stop()
            coord.join(threads)


    
    return output_labels, output_features

In [241]:
audio_output_labels = []
audio_output_features = []
audio_path = "audio_1556745450.370243.tfrecord"
audio_output_labels, audio_output_features = load_data(audio_path,
                                           'audio', audio_output_labels, audio_output_features)

video_output_labels = []
video_output_features = []
video_path = "video_1556754626.438139.tfrecord"
video_output_labels, video_output_features = load_data(video_path,
                                           'video', video_output_labels, video_output_features)

[[ 0.8784008  -1.788266    0.21957731 ... -1.9921875  -0.6588541
   2.0078125 ]
 [ 0.79996943 -1.7098346   0.29800868 ... -1.9921875  -1.1137561
   2.0078125 ]
 [ 0.8784008  -1.7412071   0.32938123 ... -1.9921875  -0.47061872
   2.0078125 ]
 ...
 [ 0.84702826 -1.7098346   0.31369495 ... -1.9921875   0.17251849
   2.0078125 ]
 [ 0.8784008  -1.7725797   0.06271458 ... -1.9921875  -0.5490502
   2.0078125 ]
 [ 0.831342   -1.7098346   0.4078126  ... -1.9921875  -0.81571686
   2.0078125 ]]
[[ 0.831342   -1.7725797  -0.03140306 ... -1.9921875  -1.9921875
   2.0078125 ]
 [ 0.90977335 -1.7098346   0.12545967 ... -1.9921875  -1.9921875
   2.0078125 ]
 [ 0.8784008  -1.7412071   0.06271458 ... -1.9921875  -1.9921875
   2.0078125 ]
 ...
 [ 0.8940871  -1.7725797  -0.03140306 ... -1.505913   -1.9921875
   2.0078125 ]
 [ 0.8940871  -1.7412071   0.14114594 ... -1.9921875  -1.9921875
   2.0078125 ]
 [ 0.8156557  -1.7412071   0.21957731 ... -1.9921875  -0.9725796
   2.0078125 ]]
[[ 1.05095    -1.7412071 

[[ 0.8784008  -1.7725797   0.17251849 ... -1.9921875  -1.2549326
   2.0078125 ]
 [ 0.8940871  -1.8196385   0.18820477 ... -1.9921875  -1.9921875
   2.0078125 ]
 [ 0.9411459  -1.788266    0.03134203 ... -1.9921875  -1.9921875
   2.0078125 ]
 ...
 [ 0.7685969  -1.788266    0.06271458 ... -1.9921875  -1.9921875
   2.0078125 ]
 [ 0.90977335 -1.7412071   0.21957731 ... -1.9921875  -1.9921875
   2.0078125 ]
 [ 0.8627145  -1.7412071   0.03134203 ... -1.9921875  -1.9921875
   2.0078125 ]]
[[ 8.9408708e-01 -1.1608149e+00  1.0666363e+00 ...  1.4431069e+00
  -3.0517578e-05  2.0078125e+00]
 [ 6.7447925e-01 -1.4588542e+00  2.6663613e-01 ...  1.4431069e+00
   3.1342030e-02  2.0078125e+00]
 [ 6.2742043e-01 -1.1921874e+00  7.9996943e-01 ... -1.9921875e+00
   9.8820472e-01  2.0078125e+00]
 ...
 [ 7.6859689e-01 -1.3019915e+00  7.8428316e-01 ...  6.1173415e-01
  -1.0196384e+00  2.0078125e+00]
 [ 8.3134198e-01 -1.4274817e+00  6.2742043e-01 ...  1.8820477e-01
  -1.7412071e+00  2.0078125e+00]
 [ 9.2545962e-

[[ 0.9568322  -1.7255208   0.07840085 ... -1.9921875  -1.9921875
   2.0078125 ]
 [ 0.9882047  -1.7098346   0.14114594 ... -1.9921875  -1.9921875
   2.0078125 ]
 [ 0.9882047  -1.7255208   0.09408712 ... -1.9921875  -1.9921875
   2.0078125 ]
 ...
 [ 1.0195775  -1.7255208   0.25094986 ... -1.9921875  -1.9921875
   2.0078125 ]
 [ 0.97251844 -1.7412071   0.31369495 ... -1.9921875  -1.9921875
   2.0078125 ]
 [ 1.05095    -1.7098346   0.31369495 ... -1.9921875  -1.9921875
   2.0078125 ]]
[[ 1.0666363  -1.678462    0.73722434 ... -1.9921875   0.79996943
   2.0078125 ]
 [ 1.0666363  -1.7255208   0.7058518  ... -1.9921875  -0.45493245
   2.0078125 ]
 [ 1.0195775  -1.7412071   0.36075377 ... -1.9921875  -1.9921875
   2.0078125 ]
 ...
 [ 1.160754   -1.3490503   1.0823226  ... -1.9921875  -0.9255208
   2.0078125 ]
 [ 1.160754   -1.1608149   1.8352637  ... -1.9921875  -1.9921875
   2.0078125 ]
 [ 1.0352638  -1.2706189   1.4117343  ... -1.5529718  -1.4117954
   2.0078125 ]]
[[ 1.0823226  -1.7568934  

[[ 0.97251844 -1.8353248  -0.15689325 ... -1.2078737  -1.3176777
   2.0078125 ]
 [ 1.0352638  -1.7098346   0.15683222 ... -0.9725796  -1.9921875
   2.0078125 ]
 [ 1.0823226  -1.8039522  -0.04708934 ... -0.5647365  -1.3647366
   2.0078125 ]
 ...
 [ 1.0195775  -1.8353248   0.20389104 ... -1.5529718  -1.9921875
   2.0078125 ]
 [ 1.003891   -1.8666973   0.01565576 ... -1.1137561  -1.0666972
   2.0078125 ]
 [ 0.90977335 -1.788266    0.2823224  ... -1.4588542  -1.9921875
   2.0078125 ]]
[[ 1.0352638  -1.7725797   0.15683222 ... -1.2549326   0.831342
   2.0078125 ]
 [ 0.78428316 -1.6314032   0.20389104 ... -1.1608149   0.15683222
   2.0078125 ]
 [ 0.97251844 -1.6000307   0.06271458 ...  0.7685969   0.39212632
   2.0078125 ]
 ...
 [ 1.1450677  -1.7098346   0.45487142 ... -0.89414823 -1.9921875
   2.0078125 ]
 [ 1.05095    -1.7255208   0.18820477 ... -1.9921875  -0.14120698
   2.0078125 ]
 [ 1.0195775  -1.678462    0.20389104 ... -1.9137561  -1.9921875
   2.0078125 ]]
[[ 0.9254596  -1.7412071  

[[ 0.8784008   0.5333028  -0.6117953  ... -0.17257953 -0.14120698
  -0.9725796 ]
 [ 1.1764402   0.5960479  -0.43924618 ...  0.29800868  2.0078125
  -0.6902267 ]
 [ 0.8627145   0.7685969  -0.04708934 ... -1.3647366  -0.15689325
  -0.94120705]
 ...
 [ 1.1764402   0.07840085  0.5333028  ... -0.2510109  -0.486305
  -0.43924618]
 [ 0.43918514  0.90977335 -0.43924618 ... -0.23532462  1.7411461
  -0.1882658 ]
 [ 1.4274206   0.8156557  -0.6745404  ...  1.3019304  -0.59610903
  -1.9137561 ]]
[[-1.8196385   0.2823224   2.0078125  ...  1.2234991   0.39212632
  -1.0039521 ]
 [ 0.09408712  0.25094986  0.5960479  ...  0.4705577  -0.70591295
  -0.40787363]
 [-0.89414823  0.07840085  2.0078125  ... -0.64316785 -0.75297177
  -0.7843443 ]
 ...
 [ 0.01565576 -0.6588541   1.8195775  ...  0.4705577   0.3450675
   0.9882047 ]
 [-0.70591295  0.48624396  2.0078125  ... -0.4235599  -0.45493245
  -1.3647366 ]
 [-0.9098345  -0.01571679  1.4117343  ...  0.8940871  -0.03140306
   0.07840085]]
[[ 0.7058518   0.3607

[[-0.83140314  0.26663613  0.7685969  ... -0.4235599  -0.34512854
  -0.6274816 ]
 [-0.89414823  0.15683222  1.1293814  ... -0.53336394 -1.1921874
  -0.3608148 ]
 [-1.0666972   0.84702826  0.5960479  ...  0.4078126   0.5803616
  -1.4588542 ]
 ...
 [-0.39218736 -0.06277561  0.26663613 ...  0.14114594 -0.51767766
   0.01565576]
 [-0.26669717 -0.2510109  -0.03140306 ...  0.72153807 -1.1765012
   0.18820477]
 [-0.20395207 -0.1255207   0.21957731 ...  0.7529106  -0.1255207
   0.15683222]]
[[-0.2510109   0.15683222  2.0078125  ...  0.61173415 -0.3608148
  -0.17257953]
 [ 0.07840085  0.7058518   1.9764402  ... -0.06277561  0.18820477
  -1.7098346 ]
 [-1.0353247   0.7529106   2.0078125  ... -0.64316785 -0.9568933
  -1.2706189 ]
 ...
 [ 0.9411459   0.3450675   1.6156559  ...  0.21957731 -0.06277561
   0.31369495]
 [-1.6470895   0.39212632  2.0078125  ...  0.84702826 -0.9098345
  -0.6588541 ]
 [-1.1608149   0.73722434  2.0078125  ...  0.4705577  -1.5215993
  -0.6902267 ]]
[[ 0.37644005 -0.8627757

[[ 1.0038910e+00  8.4702826e-01  1.8820477e-01 ...  2.0078125e+00
   1.6784010e+00 -1.9921875e+00]
 [ 3.4506750e-01  9.0977335e-01 -2.3532462e-01 ...  2.0078125e+00
   1.3333030e+00 -2.0395207e-01]
 [ 6.2714577e-02  4.2349887e-01 -3.0517578e-05 ...  2.0078125e+00
   1.2234991e+00  1.1764402e+00]
 ...
 [ 1.7251849e-01  5.9604788e-01 -6.2775612e-02 ...  2.0078125e+00
  -8.3140314e-01  6.2742043e-01]
 [-1.2552071e-01 -4.7089338e-02 -4.3924618e-01 ... -3.1403065e-02
   3.4506750e-01  2.0078125e+00]
 [ 2.1957731e-01  1.0038910e+00 -5.9610903e-01 ...  2.0078125e+00
   2.5094986e-01 -1.8196385e+00]]
[[ 0.5333028  -1.8353248   0.48624396 ...  1.0823226  -0.8470894
   0.39212632]
 [-0.34512854 -1.4902267  -0.51767766 ... -0.14120698 -0.8627757
   0.9882047 ]
 [-1.2863052  -1.6000307   0.48624396 ...  0.23526359 -0.6588541
  -0.89414823]
 ...
 [-1.1765012  -1.8980699  -0.26669717 ... -0.32944226  0.15683222
   0.5333028 ]
 [ 0.0470283  -1.505913    1.0195775  ...  0.90977335  0.2823224
   1.2391

[[-0.47061872  0.03134203  2.0078125  ...  0.1097734   0.6901655
  -0.47061872]
 [ 0.03134203  0.78428316 -0.87846196 ...  0.3450675  -1.9921875
  -1.5843444 ]
 [-1.4117954   0.43918514  1.9607539  ...  0.78428316 -0.03140306
   0.26663613]
 ...
 [-1.2706189   0.9882047  -0.28238344 ...  0.18820477  1.5372245
   0.26663613]
 [-0.7843443  -0.2510109   2.0078125  ... -0.23532462 -0.5647365
  -0.09414816]
 [ 0.831342   -0.2510109   0.9882047  ... -0.9255208  -0.32944226
  -0.89414823]]
[[-1.4117954   0.25094986  1.0823226  ... -0.07846189  1.3646755
  -0.28238344]
 [-1.0823835   0.43918514  0.8627145  ...  0.01565576  1.0352638
  -0.14120698]
 [-1.9765012   0.25094986  0.4705577  ... -0.5019914   0.56467533
  -0.10983443]
 ...
 [-0.32944226  0.15683222  0.37644005 ...  1.7725186   0.0470283
   0.7058518 ]
 [-0.17257953  0.32938123  0.56467533 ... -0.34512854  0.26663613
   1.2862442 ]
 [ 0.25094986  0.8627145  -0.81571686 ...  0.6431067   0.61173415
   1.0823226 ]]
[[ 0.32938123 -1.160814

In [242]:
audio_x = np.array(audio_output_features)
audio_y = np.array(audio_output_labels)
video_x = np.array(video_output_features)

video_y = np.array(video_output_labels)
print('audio_x shape: ', audio_x.shape)
print('audio_y shape: ', audio_y.shape)
print('video_x shape: ', video_x.shape)
print('video_y shape: ', video_y.shape)


audio_x shape:  (450, 10, 128)
audio_y shape:  (450,)
video_x shape:  (450, 10, 128)
video_y shape:  (450,)


In [244]:
print(np.sort(audio_y))
print(np.sort(video_y))

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  98  99 100 101 102 103 104 105 106 107 108 109 110 111
 112 113 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130
 131 132 133 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168
 169 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 217 218 219 220 221 222 223 224 225
 226 227 228 229 230 231 232 233 234 235 236 237 238 240 241 242 243 244
 245 246 247 248 249 250 251 252 253 254 255 256 25

In [ ]:
# --- At this point the desired data should be loaded --- 

In [384]:
def build_batch(dataset_x, dataset_y, batch_size, labels=None):
    if labels is None:
        indices = list(np.random.randint(0, len(dataset_x), size=batch_size))
    else:
        indices=[]
        for i in labels:
            #print(i)
            indices.append(np.where(dataset_y == i)[0][0])
    # Recover what the entries for the batch are
    batch_x = np.array([dataset_x[i] for i in indices])
    batch_y = np.array([dataset_y[i] for i in indices])
    
    return batch_x, batch_y, indices

In [385]:
def contrastive_loss(embedded_batch_a, embedded_batch_b, a_labels, b_labels, margin):
    y = tf.cast(tf.equal(a_labels, b_labels), tf.float32)
    dist = tf.norm(embedded_batch_a - embedded_batch_b, axis=1)
    print(dist.shape)
    loss = (y) * .5 * tf.square(dist) + (1-y) * .5 * tf.square(tf.maximum(0., margin - dist))
    return y, tf.reduce_sum(loss)

In [386]:
def build_graph(input_shape, labels_shape, 
                embed_size=128, learning_rate=0.000001, 
                l1_reg=0.001, l2_reg=0.001, margin=50):
    audio_inputs = tf.placeholder(shape=[None, input_shape[0], input_shape[1]], name="audio_inputs", dtype=tf.float32)
    video_inputs = tf.placeholder(shape=[None, input_shape[0], input_shape[1]], name="video_inputs", dtype=tf.float32)
    audio_labels = tf.placeholder(shape=[None], name="audio_labels", dtype=tf.float32)
    video_labels = tf.placeholder(shape=[None], name="video_labels", dtype=tf.float32)

    flattened_audio = tf.layers.flatten(audio_inputs)
    flattened_video = tf.layers.flatten(video_inputs)
    flattened_audio_labels = tf.layers.flatten(audio_labels)
    flattened_video_labels = tf.layers.flatten(video_labels)

    
    #subnetwork = build_fc_net(inputs, embed_size, np.prod(video_input_shape), l1_reg, l2_reg)
    weights = tf.trainable_variables()
    if l1_reg != 0:
        reg_1 = tf.contrib.layers.l1_regularizer(scale=l1_reg)
    else:
        reg_1 = 0
    if l2_reg != 0:
        reg_2 = tf.contrib.layers.l2_regularizer(scale=l2_reg)
    else:
        reg_2 = 0
    audio_embed = build_fc_net(flattened_audio, embed_size, l1_reg, l2_reg)
    video_embed = build_fc_net(flattened_video, embed_size, l1_reg, l2_reg, reuse=True)
    weights = tf.trainable_variables()

    #mse = tf.losses.mean_squared_error(audio_embed, video_embed) # only use loss if labels don't match?
    matches, error = contrastive_loss(audio_embed, video_embed, flattened_audio_labels, flattened_video_labels, margin)
    reg_penalty = tf.contrib.layers.apply_regularization(reg_1, weights) + tf.contrib.layers.apply_regularization(reg_2, weights)
    loss = error + reg_penalty
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return (audio_inputs, audio_labels,
            video_inputs, video_labels,
            loss, optimizer, matches,
            audio_embed, video_embed)

In [387]:
#audio input is a tf placeholder for the input audio features
#video labels is a tf placeholder for the input video features
#encode size is the desired size of the encoded vector (should be the same size as the video features)
#l1 and l2 reg are the amount of weight to put on l1 and l2 regularizers for the loss

def build_fc_net(input_data, embed_size, l1_reg=0.001, l2_reg=0.001, reuse=False):
    net = build_fc_layers(input_data, embed_size, reuse)
    return net

In [388]:
# input layer should be the flattened inputs
def build_fc_layers(input_layer, output_size, reuse=False):
    with tf.name_scope("model"):
        with tf.variable_scope("dense0", reuse=reuse) as scope:
            h1 = tf.layers.dense(inputs=input_layer, units=1024, activation=tf.nn.tanh)
            d1 = tf.layers.dropout(inputs=h1, rate=.3)
        with tf.variable_scope("dense1", reuse=reuse) as scope:
            h2 = tf.layers.dense(inputs=d1, units=512, activation=tf.nn.tanh)
            d2 = tf.layers.dropout(inputs=h2, rate=.3)            
        with tf.variable_scope("dense2", reuse=reuse) as scope:
            h3 = tf.layers.dense(inputs=d2, units=256, activation=tf.nn.tanh)
            d3 = tf.layers.dropout(inputs=h3, rate=.3)
        with tf.variable_scope("dense3", reuse=reuse) as scope:
            raw_encode = tf.layers.dense(inputs=d3, units=output_size, activation=tf.nn.tanh)

    return raw_encode

In [389]:
def train(sess, audio_inputs, audio_labels, 
          video_inputs, video_labels, 
          audio_x, audio_y,
          video_x, video_y,
          loss, optimizer, 
          audio_embed, video_embed, 
          batch_size, cur_epoch, num_iters=5000):

        for i in range(num_iters):
            if i % 2 == 0:
                audio_batch_input, audio_batch_label, _ = build_batch(audio_x, audio_y, batch_size)
                video_batch_input, video_batch_label, _ = build_batch(video_x, video_y, batch_size)
            else:
                audio_batch_input, audio_batch_label, indices = build_batch(audio_x, audio_y, batch_size)
                video_batch_input, video_batch_label, _ = build_batch(video_x, video_y, batch_size, audio_batch_label)                
            _, loss_val,ys = sess.run([optimizer, loss, matches], feed_dict={audio_inputs: audio_batch_input,
                                                             video_inputs: video_batch_input,
                                                             audio_labels: audio_batch_label,
                                                             video_labels: video_batch_label})
            if i % 100 == 0:
                train_summary_writer.add_scalar('train_loss', loss_val, cur_epoch * num_iters + i)
                #print(ys)
                #print(loss_val)

In [394]:
def k_nearest_neighbors(embedding, search_space, k):
    embedding_stacked = np.repeat(embedding[np.newaxis,:], len(search_space), axis=0)
    similarity = np.linalg.norm(embedding_stacked - search_space, axis=1)
    #print(similarity.shape)
    indices = np.argsort(similarity)[:k]
    #print(indices)
    return indices

def validate(sess, audio_inputs, audio_labels, 
             video_inputs, video_labels, 
             audio_x, audio_y,
             video_x, video_y,
             loss, optimizer, 
             audio_embed, video_embed, 
             cur_epoch, batch_size=100, k=1):

    audio_batch_input, audio_batch_label, indices = build_batch(audio_x, audio_y, batch_size)
    #print(audio_batch_label)
    video_batch_input, video_batch_label, _ = build_batch(video_x, video_y, batch_size, audio_batch_label)
    #print(video_batch_label)
    audio_embeddings = sess.run(audio_embed, feed_dict={audio_inputs: audio_batch_input,
                                                          audio_labels: audio_batch_label})
    video_embeddings = sess.run(video_embed, feed_dict={video_inputs: video_batch_input,
                                                          video_labels: video_batch_label})
    matches = 0
    for i in range(len(audio_embeddings)):
        a = audio_embeddings[i]
        if i%10 == 0:
            print(a)
        a_y = audio_batch_label[i]
        ids = k_nearest_neighbors(a, video_embeddings, k)
        #print(a_y)
        #print(ids)
        if a_y in video_batch_label[ids]:
            matches += 1
    train_summary_writer.add_scalar('found_matches', matches/batch_size, cur_epoch)

In [ ]:
train_log_dir = 'logs/tensorboard/train/log2'
train_summary_writer = SummaryWriter(train_log_dir)


num_epochs = 20
batch = 50
m=.5

tf.reset_default_graph()

(audio_inputs, audio_labels, 
 video_inputs, video_labels, 
 loss, optimizer, matches,
 audio_embed, video_embed) = build_graph(audio_x[0].shape, audio_y.shape[0], margin=m)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(num_epochs):
    validate(sess, audio_inputs, audio_labels,
          video_inputs, video_labels,
          audio_x, audio_y, 
          video_x, video_y, 
          loss, optimizer, 
          audio_embed, video_embed, epoch)

    train(sess, audio_inputs, audio_labels,
          video_inputs, video_labels,
          audio_x, audio_y, 
          video_x, video_y, 
          loss, optimizer, 
          audio_embed, video_embed, batch, epoch)



(?,)
[ 0.44535944  0.28898397  0.7700776   0.5364985   0.2605488  -0.24467075
  0.15468624 -0.29923743  0.17556348  0.08975634 -0.1988028   0.51032305
  0.20581988 -0.0556705   0.8161454   0.09789355 -0.24764355  0.15892251
 -0.5775224  -0.3562003   0.6865728  -0.25786462 -0.19780272  0.37964076
 -0.15605512 -0.00657549 -0.18759076  0.35691398  0.24879016  0.58285165
  0.11134934 -0.53184867 -0.02068758 -0.4412937  -0.66180193  0.7390685
  0.5136087  -0.43226138  0.26414746  0.28413203 -0.27051947 -0.22488528
  0.63540584 -0.20086461  0.44890907 -0.6318565  -0.6204419  -0.61262906
 -0.13148211 -0.67146635 -0.2556248   0.2853785  -0.04663495  0.5910573
 -0.2829409   0.5367718  -0.39167538  0.09892389  0.15303919  0.49195227
 -0.35716438  0.6041343   0.7280226   0.35182616 -0.4338565  -0.7206281
  0.26236916  0.8900981   0.263205   -0.2346559  -0.30573666 -0.1853896
  0.21238983 -0.46473318  0.43208042 -0.7995485  -0.9109812   0.62379354
  0.4958568  -0.12299341 -0.3384395   0.0377471   

In [ ]:
validate(sess, audio_inputs, audio_labels,
      video_inputs, video_labels,
      audio_x, audio_y, 
      video_x, video_y, 
      loss, optimizer, 
      audio_embed, video_embed, epoch)